# Demoing a GPT-2 model finetuned on 28,000 tweets from the @realdonaldtrump twitter account

This notebook demontrates how to build a bot around a finetuned GPT-2 language model. The code below is similar to what underpins the [roboTrump Twitter bot](https://twitter.com/RoboTrump3).
This notebook is also intended for [Google Colab](https://colab.research.google.com/). Some commands may not work elsewhere.
To run this notebook, you need a GPT-2 model. You can finetune one yourself using the [Trump_finetune.ipynb](https://github.com/aaronbrezel/GPT-2_Demo/blob/master/Trump_finetune.ipynb). Or you can use a pretrained model by downloading this [checkpoint folder](https://github.com/aaronbrezel/GPT-2_Demo/tree/master/bot/checkpoint) and uploading it to your Google Drive.


## Set up coding environment to run the bot

This is same process we used in the finetuning notebook

In [1]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

     |████████████████████████████████| 645kB 14.0MB/s 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
%tensorflow_version 1.x
import tensorflow as tf
assert tf.__version__ <= "2.0"

Mount your gdrive where the .tar model checkpoint is stored. This allows us to seamlessly load the model.

In [3]:
from google.colab import drive
drive.mount('/content/drive') #For the record, there is also a method in gpt-2-simple that does this, but it's the same action

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


The next code snippet copies and unzips the .tar file from you drive. It creates a "checkpoint" folder. Inside is another folder named for the run_name that you established in the previous notebook. Inside that is all the files that you need to run the generate method. With them, we can make the bot talk.  

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name="trump_tune_small") #Use the same run name you picked in the previous notebook

Establish a gpt-2 session and load the run

In [5]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name="trump_tune_small")

Loading checkpoint checkpoint/trump_tune_small/model-400
INFO:tensorflow:Restoring parameters from checkpoint/trump_tune_small/model-400


### Set methods for running the bot

gpt_predict will be our method for generating text off a prompt and cleaning it up to look like a tweet

In [0]:
import random
def gpt_predict(prompt):

  gen_text = gpt2.generate(sess, length=250, temperature=0.7, prefix=prompt, run_name='trump_tune_small', return_as_list=True)[0]

  text_minus_tags = str(gen_text).replace("<|endoftext|>","").replace("<|startoftext|>", "") #remove text artifacts from gpt-2 output

  short_tweet_text = shorten_output(text_minus_tags) #shorten output of GPT-2 into a useable tweet length 
  short_tweet_tokens = remove_outbound_links_and_blank_tokens(short_tweet_text) #remove any outbound links
  short_tweet_text_no_clause = remove_hanging_clause(short_tweet_tokens) #remove text at the end that does not form a whole sentence


  return short_tweet_text_no_clause

def shorten_output(text):
  distribution = [1,1,1,1,1,1,1,1,1,1,1,2,2,2,3] #73 percent chance of selecting a tweet thread of length 1. 20 percent of length 2. ~7 percent for 3

  max_tweet_length = int(280*random.choice(distribution)) #Sets the max length our generated text can be given a determined number of tweets in the thread
  random_length_generator = random.randrange(30) #We don't want our tweets to always fill all 280 characters so we add a little more randomness to the length

  return text[:max_tweet_length-random_length_generator] 

def remove_outbound_links_and_blank_tokens(text):
  token_list = text.split(" ")

  for index,token in enumerate(token_list):  
    if token[:12] == "https://t.co" or token[:11] == "http://t.co":
      token_list.pop(index) #removes outbound links
    elif token == "":
      token_list.pop(index) #removes blank tokens
    elif token == "&amp;": #output not recognizing ampersand
      token_list[index] = "&"

  return token_list

def remove_hanging_clause(token_list): #Makes sure the tweet ends in a full sentence
  latest_terminator_index = 0
  #print(token_list)
  for index,token in enumerate(token_list): 
    if token[-1] == "." or token[-1] == "!": #checks to see if a word ends in a terminator, used to keep the tweet to full sentences
      latest_terminator_index = index

  #print(latest_terminator_index)
  shorten_tokens = token_list[:latest_terminator_index+1]
  return " ".join(shorten_tokens)

Test some output!

In [47]:
output = gpt_predict("This is a witchhunt")

print(output)

['This', 'is', 'a', 'witchhunt!\n“I’ve', 'had', 'a', 'very', 'good', 'meeting', 'with', 'Mitch', 'McConnell', 'and', 'the', 'Senate', 'Republicans.', 'A', 'lot', 'of', 'good', 'things', 'will', 'happen', 'on', 'our', 'agenda.”', '@McConnellsen', 'Just', 'spoke', 'to', 'Mitch', 'McConnell', 'and', 'others.', 'Big', 'win', 'on', 'Repeal', '&amp;', 'Replace.', 'We', 'will', 'get', 'it', 'done!\n“The', 'R']
39
This is a witchhunt!
“I’ve had a very good meeting with Mitch McConnell and the Senate Republicans. A lot of good things will happen on our agenda.” @McConnellsen Just spoke to Mitch McConnell and others. Big win on Repeal &amp; Replace.


**Side note**: there's a few edge cases I haven't cleaned up yet, so the cell above may through an error every once in a while. Just run again if that happens.

### Make the bot

We define a function that will run in a loop. This runs the bot

In [0]:
def prompt_predict():
  
  print("Hello! Let's get started")
  print("Input a sentence or two into the input field and watch the program complete the paragraph")
  print("")
  print("type 'quit' to exit the program")

  while True:

    prompt = input("> ")
    print(gpt_predict(prompt))

    if prompt == "quit":
      break

Now have fun! 

In [0]:
prompt_predict()